In [48]:
# import necessary libraries
import torch
import torchvision
import torch.nn.functional as F
import torchvision.transforms as T
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
import os
from os import listdir
from os.path import isfile, join
from torch.utils.tensorboard import SummaryWriter

# print cuda info
print(f"Cuda available: {torch.cuda.is_available()}")
print(f"Cuda device count: {torch.cuda.device_count()}")

Cuda available: True
Cuda device count: 1


In [49]:
def setupLabelsDict(annotations_frame):
   
    labels = {}
    index = 0
    for i in list(annotations_frame):
        if(i != "id"):
            for j in range(min(annotations_frame[i]), max(annotations_frame[i])+1):
                labels[f"{i}-{j}"] = index
                index+=1
    return labels

def getTargetEncoding(id, annotations_frame, labels):
    encoding = [0 for _ in range(len(labels))]
    labels_df = annotations_frame.loc[annotations_frame['id'] == id]
    for label, content in labels_df.items():
        if(label != 'id'):
            encoding[labels["%s-%s" % (label, labels_df[label].iloc[0])]] += 1
    return encoding

def convertAnnotationsFrame(annotations_frame, train_dir):
    annotations_frame = pd.read_csv('dataset/annotations_train.csv')

    img_files = [f for f in listdir(train_dir)]

    augmented_annotations_list = [] 
    for entry in annotations_frame.itertuples():
        for i in img_files:
            if(int(entry[1]) == int(i.split("_")[0])):
                img_with_annotation = {
                    "id": i, 
                    "age": entry[2], 
                    "backpack":entry[3],
                    "bag":entry[4],
                    "handbag":entry[5],
                    "clothes":entry[6],
                    "down":entry[7],
                    "up":entry[8],
                    "hair":entry[9],
                    "hat":entry[10],
                    "gender":entry[11],
                    "upblack":entry[12],
                    "upwhite":entry[13],
                    "upred":entry[14],
                    "uppurple":entry[15],
                    "upyellow":entry[16],
                    "upgray":entry[17],
                    "upblue":entry[18],
                    "upgreen":entry[19],
                    "downblack":entry[20],
                    "downwhite":entry[21],
                    "downpink":entry[22],
                    "downpurple":entry[23],
                    "downyellow":entry[24],
                    "downgray":entry[25],
                    "downblue":entry[26],
                    "downgreen":entry[27],
                    "downbrown":entry[28]
                }
                augmented_annotations_list.append(img_with_annotation)

    augmented_annotations_frame = pd.DataFrame(augmented_annotations_list)
    return augmented_annotations_frame


In [50]:
class PeopleDataset(Dataset):
    """People with annotations dataset."""

    def __init__(self, frame_with_labels, root_dir, labels, train, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.annotations_frame = frame_with_labels
        self.root_dir = root_dir
        self.transform = transform
        self.img_files = [f for f in listdir(root_dir)]
        self.labels = labels
        self.train = train
    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        if self.train:
            img_name = os.path.join(self.root_dir,self.annotations_frame.iloc[idx, 0])
            image = io.imread(img_name)
            image = T.ToTensor()(image)
            image = F.interpolate(image, size=128)  
            encoding = getTargetEncoding(self.annotations_frame.iloc[idx, 0],self.annotations_frame, self.labels)
            sample = (image, torch.tensor(encoding))
            return sample
        else:
            image = io.imread("%s/%s" % (self.root_dir, self.img_files[idx]))
            image = T.ToTensor()(image)
            image = F.interpolate(image, size=128)  
            sample = image
            return sample

In [51]:
'''
Input arguments
  num_classes: number of classes in the dataset.
               This is equal to the number of output neurons.
'''

def initialize_alexnet(num_classes):
  # load the pre-trained Alexnet
  alexnet = torchvision.models.alexnet(pretrained=True)
  
  # get the number of neurons in the penultimate layer
  in_features = alexnet.classifier[6].in_features
  
  # re-initalize the output layer
  alexnet.classifier[6] = torch.nn.Sequential(
    torch.nn.Linear(in_features=in_features, out_features=num_classes),
    torch.nn.Sigmoid()
  )
  return alexnet

In [52]:
def get_data(labels, batch_size, img_root, test_batch_size=256):
  
  # Prepare data transformations and then combine them sequentially
  # transform = list()
  # transform.append(T.ToTensor())                            # converts Numpy to Pytorch Tensor
  # transform.append(T.Normalize(mean=[0.5], std=[0.5]))      # Normalizes the Tensors between [-1, 1]
  # transform = T.Compose(transform)                          # Composes the above transformations into one.

  # Load data
  test_data = PeopleDataset(frame_with_labels=None,
                                      root_dir="%s/test" % (img_root),
                                      labels=labels,
                                      train=False)
  
  test_loader = torch.utils.data.DataLoader(test_data, test_batch_size, shuffle=False, num_workers=0) #before num_workers=4
  
  return test_loader

In [60]:
def test(net, data_loader, device='cuda:0'):
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.

  net.eval() # Strictly needed if network contains layers which has different behaviours between train and test
  with torch.no_grad():
    for batch_idx, inputs in enumerate(data_loader):
      print(batch_idx)
      # Load data into GPU
      inputs = inputs.to(device)
      
      outputs = net(inputs)
      #print(outputs)
      # Apply the loss
      predicted = torch.round(outputs)
      print(predicted)
  return predicted


In [54]:
def get_cost_function():
  cost_function = torch.nn.BCELoss()
  return cost_function

In [61]:
def main(device='cuda:0', 
         img_root='./dataset',
         batch_size=128):
  
  writer = SummaryWriter(log_dir="runs/exp1")

  annotations_frame = pd.read_csv("./dataset/annotations_train.csv")
  augmented_annotations_frame = convertAnnotationsFrame(annotations_frame, "%s/train" % (img_root))
  labels = setupLabelsDict(augmented_annotations_frame)
  num_classes = len(labels)
  # Instantiates dataloaders
  test_loader = get_data(labels=labels, batch_size=batch_size, img_root=img_root)
  
  # Instantiates the model
  net = initialize_alexnet(num_classes)
  net.load_state_dict(torch.load("./model.pth"))
  net.to(device)
  print('Before training:')
  test_res = test(net, test_loader)
  print("Test result: %s " % (test_res))
  print('-----------------------------------------------------')

In [62]:
main()

Before training:
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
